In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/y_train.npy
/kaggle/input/X_test.npy
/kaggle/input/X_train.npy


In [2]:
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline
%config lnlineBackend.figure_format = 'retina'

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import keras

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.optimizers import RMSprop, Adam
from keras.layers import Conv2D, MaxPooling2D, Flatten   # required for CNN
from keras import backend as K
from keras import losses
from keras import regularizers
from keras.layers import GaussianNoise


Using TensorFlow backend.


In [17]:
# load the raw data
X_train = np.load('../input/X_train.npy')
y_train = np.load('../input/y_train.npy')
print('shape of the training data:', X_train.shape)
print('shape of the target data:', y_train.shape)

shape of the training data: (3313, 150, 150)
shape of the target data: (3313,)


In [18]:
img_rows, img_cols=150,150
x_train, lable_train=X_train[:2648], y_train[:2648]
lable_train=lable_train-1
X_test, y_test=X_train[2648:], y_train[2648:]
y_test-=1
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)


X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
lable_train = keras.utils.to_categorical(lable_train, 4)
y_test = keras.utils.to_categorical(y_test, 4)

In [64]:
noise_salt = np.random.randint(-1, 1, (img_rows, img_cols))
noise_pepper = np.random.randint(-1, 1, (img_rows, img_cols))


In [68]:
noise_salt = noise_salt.reshape(img_rows, img_cols, 1)
noise_pepper = noise_pepper.reshape(img_rows, img_cols, 1)
xnew=np.append(x_train,x_train[:150]+noise_salt+noise_pepper,axis=0)
ynew=np.append(lable_train,lable_train[:150],axis=0)

In [69]:
model = Sequential()
model.add(Conv2D(128,# number of kernels 
                kernel_size=(3, 3),activation='relu',input_shape=(150,150,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))#0.25
model.add(Flatten())
model.add(Dense(128, activation='relu',kernel_regularizer=regularizers.l2(0.02)))
model.add(Dropout(0.25))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(4, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_40 (Conv2D)           (None, 148, 148, 128)     1280      
_________________________________________________________________
max_pooling2d_40 (MaxPooling (None, 74, 74, 128)       0         
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 72, 72, 64)        73792     
_________________________________________________________________
max_pooling2d_41 (MaxPooling (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 34, 34, 32)        18464     
_________________________________________________________________
max_pooling2d_42 (MaxPooling (None, 17, 17, 32)        0         
_________________________________________________________________
dropout_40 (Dropout)         (None, 17, 17, 32)      

In [70]:
batch_size = 64
epochs = 100

from keras.callbacks import EarlyStopping
early_stopping_monitor = EarlyStopping(monitor='val_loss', patience=8)

history = model.fit(xnew, ynew,validation_split=0.5,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          callbacks=[early_stopping_monitor],  
          validation_data=(X_test, y_test))


Train on 2798 samples, validate on 665 samples
Epoch 1/100
2798/2798 [==============================] - 3s 1ms/step - loss: 2.6013 - accuracy: 0.2516 - val_loss: 1.4563 - val_accuracy: 0.2677
Epoch 2/100
2798/2798 [==============================] - 3s 974us/step - loss: 1.4102 - accuracy: 0.2609 - val_loss: 1.3899 - val_accuracy: 0.3308
Epoch 3/100
2798/2798 [==============================] - 3s 970us/step - loss: 1.3877 - accuracy: 0.2991 - val_loss: 1.3879 - val_accuracy: 0.2677
Epoch 4/100
2798/2798 [==============================] - 3s 957us/step - loss: 1.3873 - accuracy: 0.2802 - val_loss: 1.3951 - val_accuracy: 0.2256
Epoch 5/100
2798/2798 [==============================] - 3s 959us/step - loss: 1.3847 - accuracy: 0.2881 - val_loss: 1.3850 - val_accuracy: 0.2647
Epoch 6/100
2798/2798 [==============================] - 3s 951us/step - loss: 1.3834 - accuracy: 0.3031 - val_loss: 1.3835 - val_accuracy: 0.3113
Epoch 7/100
2798/2798 [==============================] - 3s 947us/step - 

In [43]:
score = model.evaluate(X_test, y_test, verbose=1)
print('Test loss: {:.4}'.format(score[0]))
print('Test accuracy: {:.4}'.format(score[1]))

665/665 [==============================] - 0s 462us/step
Test loss: 0.8904
Test accuracy: 0.6421


In [44]:
def transform(result):
    y=[]
    for i in result:
        for j in (np.where(i==np.max(i))):
            for m in j:
                y.append(m+1)
    resultfinal=np.array(y)
    return resultfinal

In [45]:
X_t = np.load('../input/X_test.npy')
X_t = X_t.reshape(X_t.shape[0], img_rows, img_cols, 1)
resultfinal=transform(model.predict(X_t))
resultfinal[0]

3

In [46]:
import pandas as pd
pid=pd.DataFrame(columns=['Id'],data=np.arange(0,829,1))
category=pd.DataFrame(columns=['Category'],data=resultfinal)
testresult=pid.join(category)
testresult

Id  Category
0      0         3
1      1         1
2      2         3
3      3         4
4      4         3
..   ...       ...
824  824         3
825  825         4
826  826         1
827  827         3
828  828         4

[829 rows x 2 columns]

In [49]:
testresult.to_csv('./result2=0.6421.csv',index=False)